In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql .functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder\
         .appName('spark-ml')\
         .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 20:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet('/Users/xwyang/Desktop/sf-airbnb')

In [4]:
trainDF,testDF = df.randomSplit([0.8,0.2],seed=42)

In [5]:
numericCols = [field for (field,datatype) in df.dtypes if ((datatype=='double')& (field !='price'))]

In [6]:
veAss = VectorAssembler(inputCols=numericCols,outputCol='features')

In [7]:
lr = LinearRegression(featuresCol='features',labelCol='price',regParam=0.2)

In [8]:
pipe = Pipeline(stages=[veAss,lr])
pipeModel = pipe.fit(trainDF)

24/08/08 20:46:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/08 20:46:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [9]:
predDF = pipeModel.transform(testDF)

In [10]:
predDF.select('features','price','prediction').where(col('price')==85.0).show(10)

24/08/08 20:46:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+------------------+
|            features|price|        prediction|
+--------------------+-----+------------------+
|(26,[0,1,2,3,4,5,...| 85.0| 64.06473306263797|
|[1.0,37.78707,-12...| 85.0|175.81872773235227|
|(26,[0,1,2,3,4,5,...| 85.0|111.02067469614485|
|(26,[0,1,2,3,4,5,...| 85.0|61.926021194143686|
|(26,[0,1,2,3,4,5,...| 85.0| 176.6102637444128|
|[2.0,37.74083,-12...| 85.0|130.33326983338702|
|(26,[0,1,2,3,4,5,...| 85.0|142.82943079762845|
|(26,[0,1,2,3,4,5,...| 85.0|37.011700249691785|
|(26,[0,1,2,3,4,5,...| 85.0|3.3970729783504794|
|(26,[0,1,2,3,4,5,...| 85.0| 138.3092162393841|
+--------------------+-----+------------------+
only showing top 10 rows



In [11]:
categoryCols = [y for (y,datatype) in df.dtypes if datatype=='string']
categoryindex = [y+'Index' for y in categoryCols]
oheIndex = [y+'OHE' for y in categoryCols]

In [12]:
stringIndex = StringIndexer(inputCols=categoryCols,outputCols=categoryindex,handleInvalid='skip')

In [13]:
oheEncoder = OneHotEncoder(inputCols=categoryindex,outputCols=oheIndex)

In [14]:
assInputs = oheIndex + numericCols

In [15]:
assVect = VectorAssembler(inputCols=assInputs,outputCol='features')

In [16]:
pipeline = Pipeline(stages=[stringIndex,oheEncoder,assVect,lr])
pModel = pipeline.fit(trainDF)

In [17]:
preDF = pModel.transform(testDF)

In [18]:
preDF.select('features','price','prediction').where(col('price')==85.0).count()

20

In [19]:
predDF.select('features','price','prediction').where(col('price')==85.0).count()

20

In [20]:
predDF.select('features','price','prediction').where(col('price')==85.0).show(20)

+--------------------+-----+------------------+
|            features|price|        prediction|
+--------------------+-----+------------------+
|(26,[0,1,2,3,4,5,...| 85.0| 64.06473306263797|
|[1.0,37.78707,-12...| 85.0|175.81872773235227|
|(26,[0,1,2,3,4,5,...| 85.0|111.02067469614485|
|(26,[0,1,2,3,4,5,...| 85.0|61.926021194143686|
|(26,[0,1,2,3,4,5,...| 85.0| 176.6102637444128|
|[2.0,37.74083,-12...| 85.0|130.33326983338702|
|(26,[0,1,2,3,4,5,...| 85.0|142.82943079762845|
|(26,[0,1,2,3,4,5,...| 85.0|37.011700249691785|
|(26,[0,1,2,3,4,5,...| 85.0|3.3970729783504794|
|(26,[0,1,2,3,4,5,...| 85.0| 138.3092162393841|
|(26,[0,1,2,3,4,6,...| 85.0|-2.871833320503356|
|(26,[0,1,2,3,4,5,...| 85.0|173.68417039416454|
|[7.0,37.78555,-12...| 85.0|140.54682232525738|
|(26,[0,1,2,3,4,5,...| 85.0|57.241592022830446|
|(26,[0,1,2,3,4,6,...| 85.0|  88.3742227207913|
|(26,[0,1,2,3,4,5,...| 85.0|146.47494780067063|
|(26,[0,1,2,3,4,5,...| 85.0| 97.87276831762938|
|(26,[0,1,2,3,4,5,...| 85.0|259.41438836

In [21]:
preDF.select('features','price','prediction').where(col('price')==85.0).show(20)

+--------------------+-----+------------------+
|            features|price|        prediction|
+--------------------+-----+------------------+
|(98,[0,3,6,22,43,...| 85.0|  59.4702073803648|
|(98,[0,3,6,9,44,6...| 85.0| 203.3387047639448|
|(98,[0,3,6,35,43,...| 85.0|  92.8171582776722|
|(98,[0,3,37,43,67...| 85.0| 71.45455819377116|
|(98,[0,2,11,44,67...| 85.0|205.16452145881692|
|(98,[0,2,29,43,67...| 85.0|178.05038858991665|
|(98,[0,1,6,10,47,...| 85.0|101.15228188589174|
|(98,[0,1,6,15,43,...| 85.0| 8.507313719200909|
|(98,[0,1,6,13,42,...| 85.0|2.1055975784091743|
|(98,[0,1,6,13,42,...| 85.0| 116.0316436110212|
|(98,[0,1,6,7,42,6...| 85.0| 58.20399383738885|
|(98,[0,1,24,43,67...| 85.0|151.89176341505964|
|(98,[0,1,9,42,66,...| 85.0|158.99427028600306|
|(98,[0,4,6,12,43,...| 85.0|31.840548346108562|
|(98,[0,4,6,26,42,...| 85.0|-40.38156117120593|
|(98,[3,20,43,67,6...| 85.0|131.77360216620644|
|(98,[2,6,24,45,66...| 85.0|129.21529623668704|
|(98,[2,6,17,43,67...| 85.0|208.24525053

In [22]:
preDF.select('room_type','room_typeIndex',
 'host_is_superhostOHE','features','price','prediction').show(5)

+---------------+--------------+--------------------+--------------------+-----+------------------+
|      room_type|room_typeIndex|host_is_superhostOHE|            features|price|        prediction|
+---------------+--------------+--------------------+--------------------+-----+------------------+
|Entire home/apt|           0.0|       (1,[0],[1.0])|(98,[0,3,6,22,43,...| 85.0|  59.4702073803648|
|   Private room|           1.0|       (1,[0],[1.0])|(98,[0,3,6,22,43,...| 45.0|26.087910872442023|
|   Private room|           1.0|       (1,[0],[1.0])|(98,[0,3,6,22,43,...| 70.0| 30.62218454249188|
|   Private room|           1.0|       (1,[0],[1.0])|(98,[0,3,6,12,42,...|128.0|-92.16468853100145|
|   Private room|           1.0|       (1,[0],[1.0])|(98,[0,3,6,12,43,...|159.0| 94.25964568378333|
+---------------+--------------+--------------------+--------------------+-----+------------------+
only showing top 5 rows



In [23]:
from pyspark.ml.feature import RFormula

In [24]:
rFormula = RFormula(formula='price ~ .',
                   featuresCol='features',
                   labelCol='price',
                   handleInvalid='skip')

In [25]:
pp = Pipeline(stages=[rFormula,lr])
ppModel = pp.fit(trainDF)

In [26]:
ppDF = ppModel.transform(testDF)

In [27]:
ppDF.select('room_type','features','price','prediction').show(5)

+---------------+--------------------+-----+------------------+
|      room_type|            features|price|        prediction|
+---------------+--------------------+-----+------------------+
|Entire home/apt|(98,[0,3,6,7,23,4...| 85.0| 59.47020680987754|
|   Private room|(98,[0,3,6,7,23,4...| 45.0|26.087910531152602|
|   Private room|(98,[0,3,6,7,23,4...| 70.0|30.622183194288482|
|   Private room|(98,[0,3,6,7,13,4...|128.0|-92.16468818696768|
|   Private room|(98,[0,3,6,7,13,4...|159.0| 94.25964596688505|
+---------------+--------------------+-----+------------------+
only showing top 5 rows



In [28]:
ppModel.stages[-1]

LinearRegressionModel: uid=LinearRegression_fe0777a528be, numFeatures=98

In [29]:
ppModel.stages[-1].intercept

393.8279892942715

In [30]:
ppModel.stages[-1].coefficients

DenseVector([-10.3818, -1028.076, -1039.2248, -1023.8434, -1092.0643, -1038.6517, -5.9706, -0.0566, 50.5815, 34.0943, 4.9171, 2.8371, 44.8145, 23.4878, 13.8886, 54.7334, 31.2028, 16.5577, 2.6317, -28.9796, 8.8263, 31.887, 11.7916, -11.7544, 42.1266, 64.3939, -43.3159, -12.2148, 105.0213, 5.5051, 122.3288, 56.5261, -44.7502, 85.6137, 31.8574, 98.8253, 37.6499, 74.9841, 76.3703, 76.8176, 215.0816, 40.464, 56.0555, 2411.5549, 740.2634, -35.1491, 3.257, 31.535, -18.749, 5.0082, -14.2747, 2.7012, -33.1626, 37.9367, -68.7046, -97.6082, 2.8896, -21.1477, -8.8344, -5.576, 38.0388, -2.2063, 110.7228, -96.2194, -13.8328, -106.9182, -81.3161, 70.4443, 416.6028, 104.6285, 50.5914, 27.7728, 20.1806, 59.1793, 1.6611, -15.0608, -5.9146, -32.4626, -6.8911, -1.3097, -0.1906, 1.5349, -0.9362, 25.1337, 1.1127, -7.0245, 8.7914, -25.6791, -55.2794, -12.7712, -6.4162, 52.5129, 39.785, 53.6548, -10.354, -93.5267, -10.354, -10.354])

In [31]:
df_index = stringIndex.fit(trainDF).transform(trainDF)

In [32]:
df_index.select('room_type','room_typeIndex','host_is_superhost','host_is_superhostIndex').show(5)

+---------------+--------------+-----------------+----------------------+
|      room_type|room_typeIndex|host_is_superhost|host_is_superhostIndex|
+---------------+--------------+-----------------+----------------------+
|   Private room|           1.0|                f|                   0.0|
|Entire home/apt|           0.0|                f|                   0.0|
|Entire home/apt|           0.0|                f|                   0.0|
|Entire home/apt|           0.0|                f|                   0.0|
|Entire home/apt|           0.0|                f|                   0.0|
+---------------+--------------+-----------------+----------------------+
only showing top 5 rows



In [33]:
encoder_df = oheEncoder.fit(df_index).transform(df_index)

In [34]:
encoder_df.select('room_type','room_typeIndex','room_typeOHE').show(5)

+---------------+--------------+-------------+
|      room_type|room_typeIndex| room_typeOHE|
+---------------+--------------+-------------+
|   Private room|           1.0|(2,[1],[1.0])|
|Entire home/apt|           0.0|(2,[0],[1.0])|
|Entire home/apt|           0.0|(2,[0],[1.0])|
|Entire home/apt|           0.0|(2,[0],[1.0])|
|Entire home/apt|           0.0|(2,[0],[1.0])|
+---------------+--------------+-------------+
only showing top 5 rows



In [35]:
final_df = assVect.transform(encoder_df)

In [36]:
model = LinearRegression(featuresCol='features',labelCol='price',regParam=0.2)
lr_model = model.fit(final_df)

In [37]:
test_index = stringIndex.fit(testDF).transform(testDF)
test_encoder = oheEncoder.fit(test_index).transform(test_index)
test_final = assVect.transform(test_encoder)

In [38]:
pDF = lr_model.transform(test_final)

In [41]:
pDF.columns

['host_is_superhost',
 'cancellation_policy',
 'instant_bookable',
 'host_total_listings_count',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'minimum_nights',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'price',
 'bedrooms_na',
 'bathrooms_na',
 'beds_na',
 'review_scores_rating_na',
 'review_scores_accuracy_na',
 'review_scores_cleanliness_na',
 'review_scores_checkin_na',
 'review_scores_communication_na',
 'review_scores_location_na',
 'review_scores_value_na',
 'host_is_superhostIndex',
 'cancellation_policyIndex',
 'instant_bookableIndex',
 'neighbourhood_cleansedIndex',
 'property_typeIndex',
 'room_typeIndex',
 'bed_typeIndex',
 'host_is_superhostOHE',
 'cancellation_policyOHE',
 'instant_bookableOHE',
 '

In [43]:
pDF.createOrReplaceTempView('sf_airbnb')

In [48]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |sf_airbnb|       true|
+---------+---------+-----------+

